In [1]:
import sys
sys.path.append('/Users/brandon/Documents/Repositories/Python/')
from FANC_auto_recon.annotations import schema_download
from FANC_auto_recon import neuroglancer_utilities
import pandas as pd
import numpy as np
from nglui import statebuilder,annotation,easyviewer,parser

In [2]:
client,token = neuroglancer_utilities.get_client()

In [3]:
neuroglancer_utilities.get_cv_path()

{'Image': {'url': 'precomputed://gs://zetta_lee_fly_vnc_001_precomputed/vnc1_full_v3align_2/realigned_v1',
  'resolution': '[4.3,4.3,45]'},
 'Flat_1': {'url': 'https://storage.googleapis.com/zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v1',
  'resolution': '[4.3,4.3,45]'},
 'FlatV_2': {'url': 'https://storage.googleapis.com/zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v2',
  'resolution': '[4.3,4.3,45]'},
 'Dynamic_V1': {'url': 'graphene://https://wclee.api.zetta.ai/segmentation/table/vnc1_full_v3align_2',
  'resolution': '[17.2,17.2,45]'},
 'Dynamic_V2': {'url': 'graphene://https://wclee.api.zetta.ai/segmentation/table/vnc1_full_v3align_2_v2',
  'resolution': '[17.2,17.2,45]'},
 'Flat_3': {'url': 'precomputed://gs://zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v3',
  'resolution': '[4.3,4.3,45]'},
 'Dynamic_V4': {'url': 'graphene://https://wclee.api.zetta.ai/segmentation/table/v

In [14]:
def find_neurons(tag, client=None, segmentation_version='Dynamic_V4', return_IDs = True, partial_match = True, search_deleted = False):
    if client is None:
        client,token = neuroglancer_utilities.get_client()
        
    tables = client.annotation.get_tables()
    annotations = pd.DataFrame(columns=['deleted', 'valid', 'schema_type', 'reference_table', 'user_id',
       'created', 'table_name', 'id', 'flat_segmentation_source',
       'description', 'tag', 'pt_position', 'superceded_id'])
    for i in tables:
        meta = client.annotation.get_table_metadata(i)
        if meta['schema_type'] == 'bound_tag':
            try:
                annotations = annotations.append(schema_download.download_annotation_table(client,i,get_deleted=search_deleted))
            except:
                print(i +' Failed')
    
    if partial_match is True:
        queried_annotations = annotations.loc[[tag in i for i in annotations.tag]]
    else:
        queried_annotations = annotations.loc[annotations.tag == tag]
        
    if len(queried_annotations) > 0:
        materialized_annotations = schema_download.generate_soma_table(queried_annotations,segmentation_version=segmentation_version)
    else:
        return('No neurons matching this query')
    
    materialized_annotations['table_name'] = queried_annotations.table_name
    if return_IDs:
        return(materialized_annotations.pt_root_id.values)

    return(materialized_annotations)

In [17]:
find_neurons('MN',return_IDs=False)

ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

In [6]:
client.annotation.get_tables()

['test_synapse_table',
 'test_synapse_table_2',
 'test_T1MN_soma_table',
 'T1MN_somas',
 '10B_somas',
 'claw_neurons',
 '81A07_Synapses',
 'test_table',
 'club_synapses',
 'claw_neuron_synapses',
 'test',
 'synapses',
 'MB_synapses',
 'Ltm synapse',
 'Mollie_synapses',
 'claw_neuron_synapses_2',
 'Hemilineages',
 '20210225 Mollie_synapses',
 'claw_neuron_synapses_3',
 'Ltm synapse-2-3',
 'tiny Ltm synapse-4-1',
 'contr-ltm-midbranch-4-1',
 'cellbody-ltm-midbranch-4-2',
 'AIZ-ltm-midbranch-4-5',
 'cellbody-ltm-midbranch-4-5',
 'tiny-ltm-4-5',
 'contra-contraltm-4-5']

In [9]:
client.annotation.get_table_metadata('T1MN_somas')['deleted'] is None

True

In [10]:
schema_download.download_annotation_table(client,'T1MN_somas')

,deleted,valid,schema_type,reference_table,user_id,created,table_name,id,flat_segmentation_source,description,tag,pt_position,superceded_id
0,None,True,NaN,NaN,NaN,2021-01-15 08:18:17.074707,T1MN_somas,1,NaN,NaN,MN_A101_T1L,"[13329, 114635, 1861]",None
1,None,True,NaN,NaN,NaN,2021-01-15 08:18:17.421020,T1MN_somas,2,NaN,NaN,MN_A101_T1R,"[63394, 114393, 2304]",None
2,None,True,NaN,NaN,NaN,2021-01-15 08:18:17.563974,T1MN_somas,3,NaN,NaN,MN_A102_T1L,"[11729, 116735, 1478]",None
3,None,True,NaN,NaN,NaN,2021-01-15 08:18:17.699061,T1MN_somas,4,NaN,NaN,MN_A102_T1R,"[63084, 116459, 2863]",None
4,None,True,NaN,NaN,NaN,2021-01-15 08:18:17.832714,T1MN_somas,5,NaN,NaN,MN_A103_T1R,"[59659, 115855, 2065]",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,None,True,NaN,NaN,NaN,2021-01-15 08:18:32.730001,T1MN_somas,113,NaN,NaN,MN_V202_T1R,"[53634, 90750, 1823]",None
56,None,True,NaN,NaN,NaN,2021-01-15 08:18:32.999174,T1MN_somas,115,NaN,NaN,MN_V401_T1L,"[17144, 89099, 2005]",None
57,None,True,NaN,NaN,NaN,2021-01-15 08:18:33.282485,T1MN_somas,117,NaN,NaN,MN_V402_T1L,"[23167, 87333, 2184]",None
58,None,True,NaN,NaN,NaN,2021-01-15 08:18:33.744803,T1MN_somas,119,NaN,NaN,MN_V501_T1R,"[54883, 84397, 2319]",None


In [ ]:
test.loc[['ee' in i for i in test.coords]]

In [ ]:
img_info = neuroglancer_utilities.get_cv_path('Image')
img_path = img_info['url']
img_res = img_info['resolution']
img_layer = statebuilder.ImageLayerConfig(img_path,name = 'Image')

seg_info_dynamic = neuroglancer_utilities.get_cv_path('Dynamic_V4')
seg_info_flat = neuroglancer_utilities.get_cv_path(version = 'Flat_3')

In [ ]:
def render_flat(img_path,seg_path,ids=None,resolution = [4.3,4.3,45]):
    img_layer = statebuilder.ImageLayerConfig(img_path,name = 'Image')
    seg_layer = statebuilder.SegmentationLayerConfig(seg_path,name='Segmentation',fixed_ids=ids)
    sb = statebuilder.StateBuilder(layers=[img_layer, seg_layer],resolution=resolution)
    return(sb.render_state(return_as='html', link_text='Flat Segmentation'))



    

In [ ]:
render_flat(img_path,seg_info_flat['url'])

In [ ]:
state = client.state.get_state_json()

In [ ]:
nglui.parser